# Use Case 10: Reactome Pathways

Enrichment analysis is often used to narrow a large set of genes and focus on those in the same pathway. Further examination of the pathway can be aided by visualization. In this use case we will demonstrate how to visualize results on Reactome Pathways.

## Step 1: Importing packages
A variety of python packages must be imported in order to download data from cptac, run statistical analysis to find meaningful clusters of proteins, and then overlay pathway diagrams found on Reactome. cptac.utils is a hosts a great source of functions for completing a variety of complicated tasks. See `help(cptac.utils)`

In [1]:
import cptac
import pandas as pd
import cptac.utils as utils
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels as statsmodels
from IPython.display import Image

cptac warning: Your version of cptac (0.9.0) is out-of-date. Latest is 0.9.1. Please run 'pip install --upgrade cptac' to update it. (/home/caleb/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py, line 1)


## Step 2: Getting the data

Even though the cptac package has been imported, we still need to load datasets. In the cell below we get the Renal Cancer proteomics data.

In [2]:
cptac.download(dataset="Ccrcc")
ds = cptac.Ccrcc()
df = ds.get_proteomics()
df.head()

Name,A1BG,A1CF,A2M,A4GALT,AAAS,AACS,AADAC,AADAT,AAED1,AAGAB,...,ZSWIM8,ZSWIM9,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
Database_ID,NP_570602.2,NP_620310.1,NP_000005.2,NP_001304967.1,NP_056480.1,NP_076417.2,NP_001077.2,NP_001273612.1,NP_714542.1,NP_078942.3,...,NP_001229416.1,NP_955373.3,NP_004715.1,NP_060445.3,NP_001005413.1,NP_001035743.1,NP_078922.1,NP_001010972.1,NP_055928.3,NP_056349.1
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00004,-0.304302,0.641447,-0.000025,NaN,0.207831,-0.364128,NaN,-1.203886,-0.217934,0.216894,...,-0.064343,NaN,-0.094441,NaN,NaN,-0.021827,0.133927,0.237280,0.114409,NaN
C3L-00010,1.195915,0.194620,1.360294,NaN,0.126956,-0.572843,NaN,-1.596546,NaN,0.221696,...,0.112064,NaN,0.072262,NaN,NaN,-0.205642,0.182434,NaN,0.201374,-0.068340
C3L-00011,-0.286155,-0.780455,-0.101089,NaN,0.292629,0.035812,NaN,NaN,NaN,0.300863,...,0.136957,NaN,0.279732,0.695116,NaN,0.316298,-0.009772,-0.019653,-0.095339,0.008961
C3L-00026,0.135730,0.404286,0.261384,NaN,0.155568,0.336311,NaN,NaN,0.709046,0.244198,...,-0.013139,NaN,0.157541,0.526188,NaN,-0.120501,0.054559,-0.313236,0.062194,0.052825
C3L-00079,-0.123959,-0.677773,-0.362547,NaN,0.187605,-0.320026,NaN,-1.300148,-0.153216,0.229676,...,-0.058953,NaN,0.152341,0.072886,0.068182,NaN,0.178869,0.266290,-0.028647,0.003682




Using utils we can get data from a variety of large databases. Here the Hugo Gene Nomenclature database is queried for a list of proteins in the proteosome. 



In [3]:
hgnc_lists = utils.get_hgnc_protein_lists()
proteasome_proteins = sorted(set(hgnc_lists["Proteasome"]))
included_cols = (df.columns.get_level_values(0) & proteasome_proteins).drop_duplicates()
included_cols

/home/caleb/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  This is separate from the ipykernel package so we can avoid doing imports until


Index(['ADRM1', 'PSMA1', 'PSMA2', 'PSMA3', 'PSMA4', 'PSMA5', 'PSMA6', 'PSMA7',
       'PSMB1', 'PSMB10', 'PSMB2', 'PSMB3', 'PSMB4', 'PSMB5', 'PSMB6', 'PSMB7',
       'PSMB8', 'PSMB9', 'PSMC1', 'PSMC2', 'PSMC3', 'PSMC4', 'PSMC5', 'PSMC6',
       'PSMD1', 'PSMD10', 'PSMD11', 'PSMD12', 'PSMD13', 'PSMD14', 'PSMD2',
       'PSMD3', 'PSMD4', 'PSMD5', 'PSMD6', 'PSMD7', 'PSMD8', 'PSMD9', 'PSME1',
       'PSME2', 'PSME3', 'PSME4', 'PSMF1'],
      dtype='object', name='Name')

## Step 3: Identify tumor/normal differences

Next we loop throught the list of protiens, seperating tumor and normal data from each other. A t-test is then performed on both lists of data. For each protein with a p-value of less than .05, the protein name, p-value and difference between tumor mena and normal mean is stored in seperate lists.

In [4]:
nameList =[]
pVal_list = []
deltaList = []
selected_proteins = df[included_cols]

for protein in selected_proteins.columns:
    data = selected_proteins[protein]
    normal = data[data.index.str.endswith('.N')]
    tumor = data[~data.index.str.endswith('.N')]
    pVal = stats.ttest_ind(normal, tumor, equal_var = False, nan_policy='omit').pvalue 
    
    if pVal < .05:
        name = protein[0]
        delta = tumor.mean() - normal.mean()
        nameList.append(name)
        pVal_list.append(pVal)
        deltaList.append(delta)
        nameList
     
print(nameList)

['PSMA1', 'PSMA2', 'PSMA3', 'PSMA4', 'PSMA5', 'PSMA6', 'PSMA7', 'PSMB1', 'PSMB10', 'PSMB2', 'PSMB3', 'PSMB4', 'PSMB5', 'PSMB6', 'PSMB7', 'PSMB8', 'PSMB9', 'PSMC1', 'PSMC2', 'PSMC3', 'PSMC4', 'PSMC5', 'PSMC6', 'PSMD1', 'PSMD10', 'PSMD11', 'PSMD12', 'PSMD13', 'PSMD14', 'PSMD2', 'PSMD3', 'PSMD4', 'PSMD4', 'PSMD5', 'PSMD6', 'PSMD7', 'PSMD8', 'PSME1', 'PSME2', 'PSME3', 'PSME4', 'PSMF1', 'PSMF1']


Benjamini & Hochberg/Yekutieli correction is applied to the list of p-values calculated in the previous cell. A dataframe is then constructed using the lists of names, p-values, and difference between tumor mean and normal mean. 

In [5]:
reject_null, adj_p, alpha_sidak, alpha_bonf = statsmodels.stats.multitest.multipletests(pVal_list, alpha=0.05, method='fdr_bh')
pVal_list = adj_p
zippedList =  list(zip(nameList, pVal_list, deltaList))
dfObj = pd.DataFrame(zippedList, columns =  ['Name', 'P-Value', 'Delta'])
dfObj = dfObj.set_index('Name')
print("Dataframe : " , dfObj, sep='\n')

Dataframe : 
             P-Value     Delta
Name                          
PSMA1   8.613834e-43  0.274008
PSMA2   6.293769e-37  0.277024
PSMA3   3.254115e-16  0.167801
PSMA4   2.236561e-09  0.110089
PSMA5   1.982079e-47  0.297547
PSMA6   6.504357e-32  0.252228
PSMA7   2.531697e-30  0.205970
PSMB1   5.128900e-38  0.266850
PSMB10  5.672522e-78  1.405130
PSMB2   1.266375e-26  0.213942
PSMB3   3.602553e-05  0.094346
PSMB4   1.313266e-28  0.243256
PSMB5   2.619794e-53 -1.017614
PSMB6   2.417327e-36 -0.693615
PSMB7   2.474161e-16 -0.293025
PSMB8   3.977867e-76  0.936154
PSMB9   9.735435e-89  1.235056
PSMC1   7.143526e-23  0.149478
PSMC2   1.077813e-09  0.080719
PSMC3   2.442829e-17  0.119226
PSMC4   7.590996e-12  0.094526
PSMC5   1.887491e-18  0.124820
PSMC6   1.951088e-16  0.102171
PSMD1   3.543048e-05  0.056173
PSMD10  6.808204e-78  0.493228
PSMD11  4.990214e-02  0.027130
PSMD12  1.552242e-40  0.327032
PSMD13  6.407339e-12  0.102624
PSMD14  1.997240e-12  0.098084
PSMD2   1.213373e-16  0.13

Change in abundance (Delta column) is then simplified. Any Increase is assigned 1, decrease assigned -1, no change is set at 0.

In [6]:
dfObj.insert(1, "simplified_change", dfObj["Delta"])
dfObj["simplified_change"][dfObj["Delta"] > 0] = 1
dfObj["simplified_change"][dfObj["Delta"] < 0] = -1
dfObj["simplified_change"][dfObj["Delta"] == 0] = 0

Using the Benjamini & Hochberg/Yekutieli correction function we are able to also evaluate whether or not to reject the null. This is evaluated and added as a column in the dataframe.

In [7]:
dfObj = dfObj.assign(adj_p=adj_p, reject_null=reject_null)
sig_res = dfObj[dfObj["reject_null"]]
sig_genes = dfObj.index.get_level_values(0).tolist()
sig_res

,P-Value,simplified_change,Delta,adj_p,reject_null
Name,,,,,
PSMA1,8.613834e-43,1.0,0.274008,8.613834e-43,True
PSMA2,6.293769e-37,1.0,0.277024,6.293769e-37,True
PSMA3,3.254115e-16,1.0,0.167801,3.254115e-16,True
PSMA4,2.236561e-09,1.0,0.110089,2.236561e-09,True
PSMA5,1.982079e-47,1.0,0.297547,1.982079e-47,True
PSMA6,6.504357e-32,1.0,0.252228,6.504357e-32,True
PSMA7,2.531697e-30,1.0,0.205970,2.531697e-30,True
PSMB1,5.128900e-38,1.0,0.266850,5.128900e-38,True
PSMB10,5.672522e-78,1.0,1.405130,5.672522e-78,True


## Step 4: Display data with Reactome

Within utils from the cptac package we can use a function called "reactome_pathway_overlay" to query reactome and overlay the proteosome data we have found to be significant. A detailed guide on using this function can be read by calling `help(utils.reactome_pathway_overlay)`.
In this example I have chosen to overlay my data with the R-HSA-1236978 pathway ID. Determining desired pathway ID's in relation to your specific data can be done in a variety of methods such as gene enrichment analysis.
Finally, with the quality parameter set at 10 you can zoom in on areas of the pathway to see further detail.

In [8]:
expression_vals, image_path = utils.reactome_pathway_overlay(
    'R-HSA-1236978', 
    df=dfObj['simplified_change'], 
    analysis_token=None, 
    open_browser=False,
    export_path="test.png", 
    image_format='png', 
    display_col_idx=None,                      
    diagram_colors='Modern', 
    overlay_colors='Standard', 
    quality=10)
Image(image_path)